In [1]:
corpus = [
    'This is the first document',
    'This document is the second document',
    'And this is the third one',
    'Is this the first document',
]

label = [1,1,0,0]

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('dataset_sms_spam _v1.xlsx')
# df_latih = df[100:]
corpus = df.Teks.tolist()
label = df.label.tolist()

In [4]:
# df_latih

In [5]:
# import random

In [6]:
# population = [[0,0,1,0,0,0,1],[0,1,0,0,0,0,0]]

In [7]:
# population_fitness_dictionary = {"0,0,1,0,0,0,1": 1.245, "0,1,0,0,0,0,0":1.658}

In [8]:
# def get_probability_list():
#     fitness = population_fitness_dictionary.values()
#     total_fit = float(sum(fitness))
#     relative_fitness = [f/total_fit for f in fitness]
#     probabilities = [sum(relative_fitness[:i+1]) 
#                      for i in range(len(relative_fitness))]
#     return probabilities

In [9]:
# def roulette_wheel_pop(population, probabilities, number):
#     chosen = []
#     for n in range(number):
#         r = random.random()
# #         print(r)
#         for (i, individual) in enumerate(population):
#             print(individual, r)
#             if r <= probabilities[i]:
#                 chosen.append(list(individual))
#                 break
#     return chosen

In [10]:
# for n in range(2):
#     print(n)

In [11]:
# roulette_wheel_pop(population, get_probability_list(), 2)

In [13]:
from random import randint, random
import copy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize 
import numpy as np
from sklearn.naive_bayes import MultinomialNB

def feature_extraction(data):
    i_temp = list()
    for i in data:
        i = word_tokenize(i.lower())
        i_temp += i
    return sorted(set(i_temp))

def weighting(data_, metode = "tfidf", feature=None):
    feature = sorted(set(feature))
#     print(feature)
    if metode=="tfidf":
        vectorizer = TfidfVectorizer(vocabulary = feature)
        model = vectorizer.fit(data_) 
        X = model.transform(data_)
#         fitur = vectorizer.get_feature_names()
#         vectorizer = CountVectorizer(vocabulary = feature)
#         model = vectorizer.fit(data_)  
    else:
        vectorizer = CountVectorizer(vocabulary = feature)
        model = vectorizer.fit(data_)
#         fitur = vectorizer.get_feature_names()
        X = model.transform(data_)
    return {
        "weight":X,
        "features":np.array(feature),
        "model":model
    }


def create_chromosome(min, max, length):
    chromosome = []
    for i in range(length):
        chromosome.append(randint(min, max))
    return chromosome

# def fitness(chrm, ffunc=sum):
#     return ffunc(chrm)
def get_index(data):
    d = list()
    for ix, i in enumerate(data):
        i=int(i)
        if i == 1:
            d.append(ix)
    return d

def fitness_(X,y, X_, y_, features_bin, features, alpha = 1, metode = "tfidf"):
    features = np.array(features)
    index = get_index(features_bin)
#     print(index)
#     print("00000|",features[index])
    data = weighting(X, metode = metode, feature=features[index])
    X_training = data['weight']
    clf = MultinomialNB(alpha = 1)
    clf.fit(X_training, y)
    X_ = data['model'].transform(X_)
    return clf.score(X_,y_)

def Modeling(X,y, X_, y_, features_bin, features, alpha = 1, metode = "tfidf"):
    features = np.array(features)
    index = get_index(features_bin)
#     print(index)
#     print("00000|",features[index])
    data = weighting(X, metode = metode, feature=features[index])
    X_training = data['weight']
    clf = MultinomialNB(alpha = 1)
    clf.fit(X_training, y)
    X_ = data['model'].transform(X_)
    return clf.score(X_,y_)


def create_population(nelem,  length, min=0, max=1):
    return [create_chromosome(min, max, length) for i in range(nelem)]    

# def get_fitness_and_population(population):
#     return [(fitness(x), x) for x in population]
    
# def get_roulette_wheel(population):
#     roulette_wheel = []
#     index = 0
#     for x in get_fitness_and_population(population):
#         for j in range(x[0]):
#             print(x[0])
#             roulette_wheel.append(index)
#         index += 1 
#     return roulette_wheel

def get_roulette_wheel_(population, inc=0):
    roulette_wheel = []
    index = 0
    sum_pop = sum(population)
    for x in population:
        x = int(round((x/sum_pop)*(100+inc)))
        if x< 0.6:
            x = 1
        for j in range(x):
            roulette_wheel.append(index)
        index += 1
    return roulette_wheel

import random
def select_parents(rw):
    rw_co = list(rw)
    poppp = list()
    while True:
        r1 = random.randrange(0, len(rw_co), 1)
        p1 = rw_co[r1]

    #     print(r, end=" ")
        r2 = random.randrange(0, len(rw_co), 1)
        p2 = rw_co[r2]

        if p1!=p2:
            poppp.append([p1,p2])
            rw_co = list(filter(lambda a: a != p2, rw_co))
            rw_co = list(filter(lambda a: a != p1, rw_co))

        if len(list(set(rw_co))) == 1:
            p1 = rw_co[-1]
            p2 = rw[0]
            poppp.append([p1,p2])
            return poppp
#             rw_co = list(filter(lambda a: a != p2, rw_co))
#             rw_co = list(filter(lambda a: a != p1, rw_co))

        if len(list(set(rw_co)))<=1:
            return poppp
#         return poppp


# def breed_by_crossover(parent_1, parent_2):
#     # Get length of chromosome
#     chromosome_length = len(parent_1)
    
#     # Pick crossover point, avoding ends of chromsome
#     crossover_point = random.randint(1,chromosome_length-1)
    
#     # Create children. np.hstack joins two arrays
#     child_1 = np.hstack((parent_1[0:crossover_point],
#                         parent_2[crossover_point:]))
    
#     child_2 = np.hstack((parent_2[0:crossover_point],
#                         parent_1[crossover_point:]))
    
#     # Return children
#     return child_1, child_2

def changesBinary(d):
    for ix, i in enumerate(d):
        if i == 1:
            d[ix]=0
        else:
            d[ix]=1
    return d

def crossover(pp1, pp2, panjang_fitur, jumlah_point = 7, prob_mutasi = 0.7):
#     prob_mutasi = 0.7
    titik = int(panjang_fitur/jumlah_point)
    cek = 0
    mulai = list()
    akhir = list()
    for i in range(panjang_fitur):
        if i % titik ==0:
            if cek%2==0:
                mulai.append(i)
            else:
                akhir.append(i)
            cek+=1
    for m, a in zip(mulai,akhir):
        pp1_Q = pp1[m:a]
        pp2_Q = pp2[m:a]

        pp1[m:a] = pp2_Q
        pp2[m:a] = pp1_Q

    if random.random() > prob_mutasi:
        pp1 = changesBinary(pp1)

    if random.random() > prob_mutasi:
        pp2 = changesBinary(pp2)
    return [pp1, pp2]

# def get_fitness_child(ortu, corpus_latih,label_latih, corpusU,labelU, fitur, alpha = 1, metode = "tfidf", pop=pop):
#     output = list()
#     for i in ortu:
# #         print( pop[i])
#         f = fitness_(corpus,label, corpus,label, pop[i], fitur, alpha=alpha, metode = metode)
#         output.append([pop[i], f])
#     return output

In [14]:

# features_bin
# fitur = fitur[2:]
# print(features_bin)
# fitur

In [15]:
# import nltk
# nltk.download('punkt')
# features_bin = [0, 1, 1, 1, 0, 0, 0, 1, 0]

In [18]:
# fitness_(corpus,label, corpus,label, features_bin, fitur, alpha = 1, metode = "tf")
# xx = weighting(corpus, metode = "tfidf", feature=None)

In [19]:
# round(0.0)

In [20]:
def ceck_best(best, nd):
    if nd<=best:
        return nd
    else:
        return best
    

In [21]:
# rw = get_roulette_wheel_(fitnes_pop, inc=jumlah_populasi)
# parents = select_parents(rw)

In [38]:
jumlah_populasi = 50
jumlah_point = 1000 #crossover
prob_mutasi = 0.7 #probabilitas permutasi

metode = "tfidf"
alpha = 1

target = 0.99 #target akurasi

fitur = feature_extraction(corpus)
panjang_fitur = len(fitur)

# features_bin = [1 for i in range(panjang_fitur)]

corpusU = corpus
labelU = label

pop = create_population(jumlah_populasi, panjang_fitur)

fitnes_pop = list()
temp_features_fitness = list()
for features_bin in pop:
    fns = fitness_(corpus,label, corpusU,labelU, features_bin, fitur, alpha = alpha, metode = metode)
    fitnes_pop.append(fns)
    temp_features_fitness.append([fns, features_bin])
# parents
best_fitur_list = list()
best_fitness_list = list()
best_generasi_list = list()

save_aja = list()

gen=0
while True:
    gen+=1
    print("Generasi ke",gen)
    rw = get_roulette_wheel_(fitnes_pop, inc=jumlah_populasi)
    parents = select_parents(rw)

    best_fitur_list_per_generasi = list()
    best_fitness_list_per_generasi = list()
    for p in parents:
        best_generasi_list.append(gen)
        family_fitness=list()
        family_fitur=list()

        mama_index = p[0]
        papa_index = p[1]
        
        m4ms = fitnes_pop[mama_index]
        family_fitness.append(m4ms)
        m4mr = pop[mama_index]
        family_fitur.append(m4mr)
        save_aja.append([m4ms, m4mr])
        save_aja.append([m4ms, gen, sum(m4mr),m4mr])
        
        p4ps = fitnes_pop[papa_index]
        family_fitness.append(p4ps)
        p4pr = pop[papa_index]
        family_fitur.append(p4pr)
        save_aja.append([p4ps, gen, sum(p4pr),p4pr])
        
        #crossover
        bin_mama = list(pop[mama_index])
        bin_papa = list(pop[papa_index])
        anak_binary = crossover(bin_mama, bin_papa, panjang_fitur,  jumlah_point = jumlah_point, prob_mutasi = prob_mutasi)
        
        #mencari fitness untuk child
        a1 = anak_binary[0]
        family_fitur.append(a1)
        fitness_anak_1 = fitness_(corpus,label, corpusU,labelU, a1, fitur, alpha = alpha, metode = metode)
        family_fitness.append(fitness_anak_1)
        save_aja.append([fitness_anak_1, a1])
        save_aja.append([fitness_anak_1, gen, sum(a1),a1])
        
        a2 = anak_binary[1]
        family_fitur.append(a2)
        fitness_anak_2 = fitness_(corpus,label, corpusU,labelU, a2, fitur, alpha = alpha, metode = metode)
        family_fitness.append(fitness_anak_2)
        save_aja.append([fitness_anak_2, gen, sum(a2),a2])
#         print(family_fitness)
        
        #mencari fitness terbaik untuk satu keluarga
        best_family_fitness = max(family_fitness)
        bf_index = family_fitness.index(best_family_fitness)
        best_family_fitur = family_fitur[bf_index]

        best_fitness_list.append(best_family_fitness)
        best_fitur_list.append(best_family_fitur)

        best_fitness_list_per_generasi.append(best_family_fitness)
        best_fitur_list_per_generasi.append(best_family_fitur)

        print("   ",best_family_fitness)
        print("   ","*"*50,sum(best_family_fitur),"/",len(best_family_fitur),"->",panjang_fitur,"|",gen)

    pop = list(best_fitur_list_per_generasi)
    fitnes_pop = list(best_fitness_list_per_generasi)

    if len(pop)<4 or max(best_fitness_list)>=target:
        print("="*100)
        print("Best",max(best_fitness_list))
#         print("Generasi ke-",best_generasi_list[best_fitness_list.index(max(best_fitness_list))])
        good_fitur = best_fitur_list[best_fitness_list.index(max(best_fitness_list))]
        best_fitness_list.index(max(best_fitness_list))
        print('jumlah fitur     ', sum(good_fitur))
        print('jumlah fitur asli', panjang_fitur)
        print('good_fitur')
        break

Generasi ke 1
    0.9711286089238845
    ************************************************** 2688 / 5468 -> 5468 | 1
    0.9711286089238845
    ************************************************** 2710 / 5468 -> 5468 | 1
    0.9658792650918635
    ************************************************** 2716 / 5468 -> 5468 | 1
    0.9702537182852143
    ************************************************** 2726 / 5468 -> 5468 | 1
    0.9650043744531933
    ************************************************** 2801 / 5468 -> 5468 | 1
    0.968503937007874
    ************************************************** 2706 / 5468 -> 5468 | 1
    0.9667541557305337
    ************************************************** 2739 / 5468 -> 5468 | 1
    0.9711286089238845
    ************************************************** 2780 / 5468 -> 5468 | 1
    0.9571303587051618
    ************************************************** 2700 / 5468 -> 5468 | 1
    0.9615048118985127
    ******************************************

In [229]:
# fitness_(corpus,label, corpus,label, best_fitur_list[-1], fitur, alpha = alpha, metode = metode)

In [30]:
# fitness_(corpus,label, corpus,label, save_aja[idxx][1], fitur, alpha = alpha, metode = metode)

In [39]:
fitness_(corpus,label, corpus,label, good_fitur, fitur, alpha = alpha, metode = metode)

0.9755030621172354

In [216]:
# best_generasi_list[best_fitness_list.index(max(best_fitness_list))]
sum(save_aja[idxx][1])

2705

In [24]:
s = list()
for i in save_aja:
    s.append(i[0])
idxx = s.index(max(s))
save_aja[idxx][0] 

0.9763779527559056

In [248]:
idxx

76

In [ ]:
vectorizer = TfidfVectorizer(vocabulary = feature)
X = vectorizer.fit_transform(data_)
fitur = vectorizer.get_feature_names()
vectorizer = CountVectorizer(vocabulary = fitur)
model = vectorizer.fit(data_) 

In [27]:
fitur

['!',
 '#',
 '%',
 '&',
 "'",
 "''",
 "'nissan",
 "'rp.100jt",
 "'s",
 '(',
 ')',
 '*',
 '*100*1',
 '*100*123',
 '*100*15',
 '*100*431',
 '*100*433',
 '*100*471',
 '*100*473',
 '*100*999',
 '*111*1',
 '*111*2',
 '*122*1',
 '*122*5*4',
 '*123',
 '*123*3',
 '*123*3*3',
 '*123*4*1*1',
 '*123*45',
 '*123*46',
 '*123*50',
 '*123*65',
 '*123*66',
 '*123*7887*2*5',
 '*123*88',
 '*123*888',
 '*200*200*kodeuniq',
 '*247',
 '*323*2',
 '*345',
 '*363',
 '*363*145',
 '*363*49',
 '*363*822',
 '*5094*',
 '*550*904',
 '*550*905',
 '*550*907',
 '*555*3',
 '*557*8',
 '*600',
 '*600*600',
 '*606',
 '*700*1',
 '*700*20',
 '*777',
 '*800',
 '*808*1*2*3',
 '*808*3*1*4*1',
 '*808*7',
 '*886*10000*083139195872',
 '*888',
 '*889',
 '*900',
 '*900*8',
 '*900*9',
 '*955*1',
 '*999*1',
 '*999*606',
 '*ngsih',
 '+',
 '+10gb',
 '+6281571xxxx',
 '+6287820008358',
 '+62xxxxxxxxxx',
 '+777',
 '+bonus',
 '+nelpon',
 ',',
 '-',
 '--',
 '-089664054777',
 '-1',
 '-297ccfa7',
 '-_-',
 '-dakota',
 '-davis',
 '-no',
 '-telk

2


[1, 10, 3, 4]

In [140]:
ee = a[0]
print("ee",ee)
a[0]=11
c

ee 1


[1, 10, 3, 4]

In [141]:
a

[11, 10, 3, 4]

In [142]:
ee

1

In [210]:
save_aja

[[0.9553805774278216,
  [0,
   1,
   0,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   1,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   1,


In [20]:

jumlah_populasi = 50
t_fitness = list()
t_features = list()

pop = create_population(jumlah_populasi, panjang_fitur)

fitnes_pop = list()
temp_features_fitness = list()
for features_bin in pop:
    fns = fitness_(corpus,label, corpus,label, features_bin, fitur, alpha = 1, metode = "tfidf")
#     print(fns)
    fitnes_pop.append(fns)
    temp_features_fitness.append([fns, features_bin])
    
best_per_family = list()
ix=0
while ix<10:
    rw = get_roulette_wheel_(fitnes_pop, inc=jumlah_populasi)
    parents = select_parents(rw)
    
    for p in parents:
        p1 = p[0]
        p2 = p[1]
        
        family_fitness = list()
        family_fitness.append(fitnes_pop[p1])
        family_fitness.append(fitnes_pop[p2])
        
        family_fitur = list()
        family_fitur.append(pop[p1])
        family_fitur.append(pop[p2])
        
        anak = crossover(pop[p[0]], pop[p[1]], panjang_fitur, jumlah_point = 100, prob_mutasi = 0.7)
        a1=anak[0]
        a2=anak[1]
        family_fitur.append(a1)
        family_fitur.append(a2)
        
        fitness_anak1 = fitness_(corpus,label, corpus,label, a1, fitur, alpha = 1, metode = "tfidf")
        fitness_anak2 = fitness_(corpus,label, corpus,label, a2, fitur, alpha = 1, metode = "tfidf")
        
        family_fitness.append(fitness_anak1)
        family_fitness.append(fitness_anak2)
        
        
    
    ix+=1


0.94750656167979
0.9466316710411199
0.9518810148731408
0.9440069991251093
0.9518810148731408
0.9483814523184602
0.94750656167979
0.9448818897637795
0.9553805774278216
0.9545056867891514
0.9658792650918635
0.9501312335958005
0.9413823272090989
0.94750656167979
0.9457567804024497
0.9536307961504812
0.9588801399825022
0.9440069991251093
0.9448818897637795
0.9536307961504812
0.952755905511811
0.9562554680664916
0.9518810148731408
0.94750656167979
0.9457567804024497
0.9492563429571304
0.9457567804024497
0.9518810148731408
0.9553805774278216
0.9466316710411199
0.9396325459317585
0.9483814523184602
0.9571303587051618
0.9448818897637795
0.9466316710411199
0.9536307961504812
0.9571303587051618
0.9510061242344707
0.9448818897637795
0.9536307961504812
0.9492563429571304
0.9492563429571304
0.958005249343832
0.9440069991251093
0.9562554680664916
0.9562554680664916
0.9510061242344707
0.9466316710411199
0.9518810148731408
0.9536307961504812
0.9562554680664916
0.9492563429571304
0.9501312335958005
0.9

In [39]:
# rw = get_roulette_wheel_(fitnes_pop, inc=jumlah_populasi)
# parents = select_parents(rw)
# semua[fitness_all.index(max(fitness_all))][1]
sedsad = list()
for i in semua:
    sedsad.append(i[1])
    
indexss = sedsad.index(max(sedsad))

In [ ]:
len(fitnes_pop)

In [41]:
fitness_(corpus,label, corpus,label, semua[indexss][0], fitur, alpha = 1, metode = "tfidf")

0.4978127734033246

In [ ]:
fitness_all

In [103]:
parents

[[17, 18],
 [28, 35],
 [6, 45],
 [30, 31],
 [26, 0],
 [12, 38],
 [3, 24],
 [39, 34],
 [9, 14],
 [42, 23],
 [49, 41],
 [19, 29],
 [48, 32],
 [21, 5],
 [16, 44],
 [1, 10],
 [13, 47],
 [25, 37],
 [33, 43],
 [4, 46],
 [36, 8],
 [20, 2],
 [15, 22],
 [40, 27],
 [7, 11]]

In [23]:

iss = 0
# best_fitur_t = 0
# best_fitnes_t = 0

fitur__ = list()
fitness__ = list()
# best = 0
while iss<3:
    iss+=1
    print("Generasi ke",iss)
    print(len(fitnes_pop))
    rw = get_roulette_wheel_(fitnes_pop, inc=jumlah_populasi)
    parents = select_parents(rw)
    
    for p in parents:
        p1 = p[0]
        p2 = p[1]
        fitness__.append(fitnes_pop[p1])
        fitness__.append(fitnes_pop[p2])
        
        fitur__.append(pop[p1])
        fitur__.append(pop[p2])
    
    
    
    
#     best_fitness = list()
#     best_feature = list()
#     for p in parents:
#         fitnes_family = list()
#         feature_family = list()
        
#         for ortu in p:
# #             print(ortu)
#             fitnes_family.append(fitnes_pop[ortu])
#             feature_family.append(pop[ortu])
            
            

#         anak = crossover(pop[p[0]], pop[p[1]], panjang_fitur, jumlah_point = 100, prob_mutasi = 0.7)
#         for afitur in anak:
#             feature_family.append(afitur)
#             fnst = fitness_(corpus,label, corpus,label, afitur, fitur, alpha = 1, metode = "tfidf")
#             fitnes_family.append(fnst)
#             fitur__.append(afitur)
#             fitness__.append(fnst)
        

#         best_fitness_family = max(fitnes_family)
#         index_ = fitnes_family.index(best_fitness_family)
#     #     print("index", index_, "|",len(feature_family))
#         best_feature_family = feature_family[index_]
#         print("   ",best_fitness_family)
#         best_fitness.append(best_fitness_family)
#         best_feature.append(best_feature_family)
#         print("   ","*"*70)
        
#     t_fitness += best_fitness
#     fitnes_pop = best_fitness
    
#     t_features += best_feature
#     pop = best_feature
    
    if len(pop)<=6:
#         b_ = max(fitness__)
#         print(b_)
#         index_ = fitness__.index(b_)
#         best_fitursnya = fitur__[index_]
#         print("berhenti")
        break

Generasi ke 1
200
Generasi ke 2
200
Generasi ke 3
200


In [24]:
# t_fitness.index(max(t_fitness))
# max(fitness__)
fitur__ 
fitness__

[0.952755905511811,
 0.9658792650918635,
 0.9536307961504812,
 0.9396325459317585,
 0.952755905511811,
 0.9553805774278216,
 0.9396325459317585,
 0.9545056867891514,
 0.9545056867891514,
 0.9562554680664916,
 0.9545056867891514,
 0.9606299212598425,
 0.958005249343832,
 0.9588801399825022,
 0.9571303587051618,
 0.9536307961504812,
 0.9518810148731408,
 0.9457567804024497,
 0.9510061242344707,
 0.9518810148731408,
 0.9440069991251093,
 0.9545056867891514,
 0.9606299212598425,
 0.9448818897637795,
 0.9510061242344707,
 0.9553805774278216,
 0.9518810148731408,
 0.9536307961504812,
 0.9536307961504812,
 0.9501312335958005,
 0.9501312335958005,
 0.9492563429571304,
 0.9588801399825022,
 0.9396325459317585,
 0.9536307961504812,
 0.9597550306211724,
 0.9562554680664916,
 0.9571303587051618,
 0.9413823272090989,
 0.9597550306211724,
 0.9562554680664916,
 0.9545056867891514,
 0.9571303587051618,
 0.9440069991251093,
 0.9571303587051618,
 0.9483814523184602,
 0.9606299212598425,
 0.9588801399825

In [65]:
fitness_(corpus,label, corpus,label, pop[0], fitur, alpha = 1, metode = "tdidf")

0.9641294838145232

In [26]:
fitness__.index(max(fitness__))

426

In [25]:
fitur__[0]

[0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,


In [730]:
parents

[[4, 6], [2, 3], [5, 1], [0, 0]]

In [530]:
# pop[5]

In [246]:
# random.randrange(0, 5, 1)

In [124]:
# round(0.6)

In [248]:
# for i in range(1):
#     print(i)

In [247]:
# l = list('abcdefgd')
# print(l)
# # ['a', 'b', 'c', 'd', 'e', 'f', 'g']

# l.clear('d')
# # print(l)
# # ['a', 'b', 'c', 'e', 'f', 'g']

In [684]:
# ll

In [528]:
# pp1 = [randint(0,1) for i in range(panjang_fitur)]
# pp2 = [randint(0,1) for i in range(panjang_fitur)]

# print(sum(pp1))
# print(sum(pp2))




In [529]:
# anak = crossover(pp1, pp2, panjang_fitur, jumlah_point = 7, prob_mutasi = 0.7)
# print(sum(anak[0]))
# print(sum(anak[1]))

In [401]:
# 